In [1]:
import sqlite3
import pandas as pd
import os

This notebook serves the purpose of being a display of general SQL knowledge as part of a portfolio. An e-commerce dataset will be analysed as part of this project.
[Click here for dataset link.](https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce/data)

# Dataset

In [2]:
folder_path = '/kaggle/input/brazilian-ecommerce'

# initialize empty list to store summary information
summaries = pd.DataFrame(columns=['Table Name', 'rows', 'columns', 'missing values', 'columns with missing values'])

# For each csv file, extract relevant summaries
for files in os.listdir(folder_path):
    file_path = os.path.join(folder_path, files)
    df = pd.read_csv(file_path)
    num_rows = df.shape[0]
    num_cols = df.shape[1]
    num_missing_values = df.isnull().sum().sum()
    cols_with_missing_values = df.columns[df.isnull().any()].tolist()
    summary = [files, num_rows, num_cols, num_missing_values, cols_with_missing_values]
    summaries.loc[len(summaries)]=summary

In [3]:
# display the summaries
pd.set_option('display.max_colwidth', 500)
summaries

,Table Name,rows,columns,missing values,columns with missing values
0,olist_customers_dataset.csv,99441,5,0,[]
1,olist_sellers_dataset.csv,3095,4,0,[]
2,olist_order_reviews_dataset.csv,99224,7,145903,"[review_comment_title, review_comment_message]"
3,olist_order_items_dataset.csv,112650,7,0,[]
4,olist_products_dataset.csv,32951,9,2448,"[product_category_name, product_name_lenght, product_description_lenght, product_photos_qty, product_weight_g, product_length_cm, product_height_cm, product_width_cm]"
5,olist_geolocation_dataset.csv,1000163,5,0,[]
6,product_category_name_translation.csv,71,2,0,[]
7,olist_orders_dataset.csv,99441,8,4908,"[order_approved_at, order_delivered_carrier_date, order_delivered_customer_date]"
8,olist_order_payments_dataset.csv,103886,5,0,[]


From the summary, there are three tables with missing values, the order reviews table in particular has a very high number of missing values. For each of those tables, we will be looking at their columns to determine what is the portion of missing values in them.

In [4]:
# initialize empty list to store summary information for certain tables
col_summaries=pd.DataFrame(columns=['Table Name', 'Column name', 'missing values (%)'])

# extract the portion of null values in each column of the tables
for files in os.listdir(folder_path):
    if files in ('olist_order_reviews_dataset.csv', 'olist_products_dataset.csv', 'olist_orders_dataset.csv'):
        file_path = os.path.join(folder_path, files)
        df = pd.read_csv(file_path)
        for cols in df:
            table_name=files
            column_name=cols
            nulls=100*df[cols].isnull().sum()/len(df[cols])
            
            summary = [table_name, column_name, nulls]
            col_summaries.loc[len(col_summaries)]=summary     

In [5]:
col_summaries

,Table Name,Column name,missing values (%)
0,olist_order_reviews_dataset.csv,review_id,0.000000
1,olist_order_reviews_dataset.csv,order_id,0.000000
2,olist_order_reviews_dataset.csv,review_score,0.000000
3,olist_order_reviews_dataset.csv,review_comment_title,88.341530
4,olist_order_reviews_dataset.csv,review_comment_message,58.702532
5,olist_order_reviews_dataset.csv,review_creation_date,0.000000
6,olist_order_reviews_dataset.csv,review_answer_timestamp,0.000000
7,olist_products_dataset.csv,product_id,0.000000
8,olist_products_dataset.csv,product_category_name,1.851234
9,olist_products_dataset.csv,product_name_lenght,1.851234


As seen in the summary, the null values mostly come from columns pertaining to written reviews. This may limit the ability to do nautral language processing on this dataset but will not be too relevant for exploratory data analysis.

# Export csv files to SQLite engine

In [6]:
df_customers = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_customers_dataset.csv')
df_sellers = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_sellers_dataset.csv')
df_order_reviews= pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_order_reviews_dataset.csv')
df_order_items= pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_order_items_dataset.csv')
df_products= pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_products_dataset.csv')
df_geolocation= pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_geolocation_dataset.csv')
df_category_name_translation= pd.read_csv('/kaggle/input/brazilian-ecommerce/product_category_name_translation.csv')
df_orders = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_orders_dataset.csv')
df_order_payments= pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_order_payments_dataset.csv')

In [7]:
# create a sqlite engine using sqlalchemy
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

# export dataframes to sqlite engine as tables
df_customers.to_sql("customers", con =engine)
df_sellers.to_sql("sellers", con =engine)
df_order_reviews.to_sql("order_reviews", con =engine)
df_order_items.to_sql("order_items", con =engine)
df_products.to_sql("products", con =engine)
df_geolocation.to_sql("geolocation", con =engine)
df_category_name_translation.to_sql("product_category_name_translation", con =engine)
df_orders.to_sql("orders", con =engine)
df_order_payments.to_sql("order_payments", con =engine)

103886

# SQL Analysis

In [8]:
sql='''

WITH reviews as 
(SELECT o.order_id, r.review_id, r.review_score FROM
order_reviews r JOIN orders o 
ON r.order_id=o.order_id),

order_details as
(SELECT i.seller_id, reviews.order_id, reviews.review_id, reviews.review_score FROM
reviews JOIN order_items i
ON reviews.order_id=i.order_id)

SELECT seller_id AS "Seller ID", COUNT(distinct review_id) AS "Total Reviews", 
ROUND(AVG(review_score),2) AS "Average Ratings" FROM order_details
GROUP BY seller_id
HAVING COUNT(distinct review_id)>200
ORDER BY "Average Ratings" DESC
LIMIT 10

''';

    
df_sql = pd.read_sql_query(sql,con=engine)
df_sql=df_sql.style.set_caption("Highest Rated sellers on the Olist Store")
df_sql

,Seller ID,Total Reviews,Average Ratings
0,c3cfdc648177fdbbbb35635a37472c53,282,4.450000
1,fa40cc5b934574b62717c68f3d678b6d,306,4.430000
2,7299e27ed73d2ad986de7f7c77d919fa,338,4.420000
3,a3a38f4affed601eb87a97788c949667,250,4.390000
4,fe2032dab1a61af8794248c8196565c9,290,4.380000
5,87142160b41353c4e5fca2360caf6f92,307,4.370000
6,6edacfd9f9074789dad6d62ba7950b9c,208,4.360000
7,4b9750c8ad28220fe6702d4ecb7c898f,218,4.350000
8,fa1c13f2614d7b5c4749cbc52fecda94,581,4.340000
9,0ea22c1cfbdc755f86b9b54b39c16043,234,4.330000


In [9]:
sql='''

WITH reviews as 
(SELECT o.order_id, r.review_id, r.review_score FROM
order_reviews r JOIN orders o 
ON r.order_id=o.order_id),

order_details as
(SELECT i.seller_id, reviews.order_id, reviews.review_id, reviews.review_score FROM
reviews JOIN order_items i
ON reviews.order_id=i.order_id)

SELECT seller_id AS "Seller ID", COUNT(distinct review_id) AS "Total Reviews", 
ROUND(AVG(review_score),2) AS "Average Ratings" FROM order_details
GROUP BY seller_id
HAVING COUNT(distinct review_id)>200
ORDER BY "Average Ratings" ASC
LIMIT 10

''';

    
df_sql = pd.read_sql_query(sql,con=engine)
df_sql=df_sql.style.set_caption("Lowest Rated Sellers on the Olist Store")
df_sql

,Seller ID,Total Reviews,Average Ratings
0,7c67e1448b00f6e969d365cea6b010ab,978,3.350000
1,88460e8ebdecbfecb5f9601833981930,247,3.350000
2,897060da8b9a21f655304d50fd935913,317,3.400000
3,7040e82f899a04d1b434b795a43b4617,210,3.530000
4,1835b56ce799e6a4dc4eddc053f04066,421,3.590000
5,d2374cbcbb3ca4ab1086534108cc3ab7,526,3.640000
6,855668e0971d4dfd7bef1b6a4133b41b,309,3.730000
7,dc4a0fc896dc34b0d5bfec8438291c80,239,3.740000
8,4c2b230173bb36f9b240f2b8ac11786e,258,3.760000
9,70a12e78e608ac31179aea7f8422044b,313,3.770000


In [10]:
sql='''

WITH order_details as
(SELECT i.order_id, i.seller_id, i.price AS Revenue, o.order_purchase_timestamp
FROM order_items i join orders o
ON i.order_id=o.order_id),

running_total_revenue AS
(SELECT seller_id, order_purchase_timestamp,
SUM(Revenue) OVER (PARTITION BY seller_id ORDER BY order_purchase_timestamp) AS rev_run_total,
SUM(Revenue) OVER (PARTITION BY seller_id) AS current_rev
FROM order_details)

SELECT seller_id as "Seller ID", MIN(order_purchase_timestamp) as "Date Achieved $100k",  current_rev as "Current Total Revenue" 
FROM running_total_revenue r
WHERE rev_run_total >=100000
GROUP BY seller_id
ORDER by "Date Achieved $100k" ASC


''';

df_sql = pd.read_sql_query(sql,con=engine)
df_sql=df_sql.style.set_caption("Fastest Sellers to Achieve $100k in sales")
df_sql

,Seller ID,Date Achieved $100k,Current Total Revenue
0,7e93a43ef30c4f03f38b393420bc753a,2017-09-01 13:13:18,176431.870000
1,53243585a1d6dc2643021fd1853d8905,2017-10-01 19:27:14,222776.050000
2,4a3ca9315b744ce9f8e9374361493884,2017-11-07 12:21:51,200472.920000
3,46dc3b2cc0980fb8ec44634e21d2718e,2017-12-04 15:38:16,128111.190000
4,fa1c13f2614d7b5c4749cbc52fecda94,2018-01-06 09:54:49,194042.030000
5,7a67c85e85bb2ce8582c35f2203ad736,2018-01-09 12:02:51,141745.530000
6,7c67e1448b00f6e969d365cea6b010ab,2018-01-13 21:57:41,187923.890000
7,4869f7a5dfa277a7dca6462dcf3b52b2,2018-01-18 10:01:14,229472.630000
8,da8622b14eb17ae2831f4ac5b9dab84a,2018-02-26 16:43:25,160236.570000
9,5dceca129747e92ff8ef7a997dc4f8ca,2018-04-19 10:10:12,112155.530000


In [11]:
sql='''

WITH order_details as 
(SELECT i.order_id, o.customer_id, o.order_purchase_timestamp, i.price AS revenue
FROM order_items i join orders o
ON i.order_id=o.order_id),

customer_details as
(SELECT o.customer_id, o.order_id, CONCAT(strftime('%Y', o.order_purchase_timestamp),'-',strftime('%m', order_purchase_timestamp)) as purchase_month, o.revenue, c.customer_city, c.customer_state
FROM customers c join order_details o
ON c.customer_id=o.customer_id),

cities_ranked as
(SELECT purchase_month, customer_city, customer_state, 
ROW_NUMBER() OVER (PARTITION BY purchase_month ORDER BY revenue DESC) AS rank FROM
customer_details 
GROUP BY purchase_month, customer_city
)

SELECT purchase_month as "Month", 
MAX(CASE WHEN rank = 1 THEN customer_city END) AS "Top City",
MAX(CASE WHEN rank = 2 THEN customer_city END) AS "Second City",
MAX(CASE WHEN rank = 3 THEN customer_city END) AS "Third City",
MAX(CASE WHEN rank = 4 THEN customer_city END) AS "Fourth City",
MAX(CASE WHEN rank = 5 THEN customer_city END) AS "Fifth City"
FROM cities_ranked
GROUP BY purchase_month

''';


df_sql = pd.read_sql_query(sql,con=engine)
df_sql=df_sql.style.set_caption("Cities with the Most Revenue by Month")
df_sql

,Month,Top City,Second City,Third City,Fourth City,Fifth City
0,2016-09,passo fundo,sao joaquim da barra,boa vista,None,None
1,2016-10,quissama,vargem grande paulista,santo antonio da patrulha,itaquaquecetuba,gravatal
2,2016-12,curitiba,None,None,None,None
3,2017-01,quarai,trindade do sul,santa fe de goias,sao joao do itaperiu,vila velha
4,2017-02,campo grande,tailandia,araras,campestre da serra,santana do livramento
5,2017-03,monte alegre do sul,pianco,itapiranga,taio,ibitita
6,2017-04,araruama,valente,cascavel,patrocinio paulista,arapiraca
7,2017-05,mandaguacu,jatai,petrolina,cunha,barra mansa
8,2017-06,pacatuba,gurupi,sao pedro,porto seguro,abaetetuba
9,2017-07,lucas do rio verde,paranatinga,vitorino freire,querencia,russas


In [12]:
sql='''

WITH product_details as
(SELECT product_id, product_category_name_english AS product_category
FROM products p join product_category_name_translation t
ON p.product_category_name=t.product_category_name),

order_details as
(SELECT o.order_id, o.product_id, p.product_category, o.price
FROM product_details p join order_items o
ON p.product_id=o.product_id)

SELECT product_category, SUM(price) AS revenue, 100*SUM(price)/(SELECT SUM(price) FROM order_details) AS percent_revenue
FROM order_details 
GROUP BY product_category ORDER BY percent_revenue DESC
LIMIT 10

''';


df_sql = pd.read_sql_query(sql,con=engine)
df_sql=df_sql.style.set_caption("Revenue and Percentage of Revenue by Product Category")
df_sql


,product_category,revenue,percent_revenue
0,health_beauty,1258681.340000,9.388524
1,watches_gifts,1205005.680000,8.988157
2,bed_bath_table,1036988.680000,7.734915
3,sports_leisure,988048.970000,7.369873
4,computers_accessories,911954.320000,6.802282
5,furniture_decor,729762.490000,5.443310
6,cool_stuff,635290.850000,4.738645
7,housewares,632248.660000,4.715953
8,auto,592720.110000,4.421109
9,garden_tools,485256.460000,3.619536


In [13]:
sql='''

SELECT DISTINCT
seller_id, SUM(price) OVER (PARTITION BY seller_id) AS sales,
COUNT(order_id) OVER (PARTITION BY seller_id) AS orders_fulfilled,
100*(SUM(price) OVER (PARTITION BY seller_id) / (SELECT SUM(price) FROM order_items)) AS market_share
FROM order_items
WHERE seller_id IS NOT NULL
ORDER BY sales DESC
LIMIT 20

''';

df_sql = pd.read_sql_query(sql,con=engine)
df_sql=df_sql.style.set_caption("Sellers by Sales, Orders, and Total Market Share of Sales")
df_sql


,seller_id,sales,orders_fulfilled,market_share
0,4869f7a5dfa277a7dca6462dcf3b52b2,229472.630000,1156,1.688336
1,53243585a1d6dc2643021fd1853d8905,222776.050000,410,1.639066
2,4a3ca9315b744ce9f8e9374361493884,200472.920000,1987,1.474972
3,fa1c13f2614d7b5c4749cbc52fecda94,194042.030000,586,1.427657
4,7c67e1448b00f6e969d365cea6b010ab,187923.890000,1364,1.382643
5,7e93a43ef30c4f03f38b393420bc753a,176431.870000,340,1.298091
6,da8622b14eb17ae2831f4ac5b9dab84a,160236.570000,1551,1.178934
7,7a67c85e85bb2ce8582c35f2203ad736,141745.530000,1171,1.042887
8,1025f0e2d44d7041d6cf58b6550e0bfa,138968.550000,1428,1.022456
9,955fee9216a65b617aa5c0531780ce60,135171.700000,1499,0.994521
